# Übungszettel 5: Clustering

## Maschinelles Lernen - WiSe 23/24

### Abgabe 22.11.2023, 23:55 Uhr

*Hinweise:*
- Übungsaufgaben **müssen** in Gruppen von 3-4 Personen abgegeben werden. **Einzelabgaben werden nicht korrigiert bzw. bewertet.**
- Es wird pro Übungszettel nur eine Aufgabe bewertet, die übrigen Aufgaben dienen zur selbstständigen Vertiefung des Vorlesungsstoffs. Für diese Aufgaben werden nach der Abgabe Musterlösungen bereitgestellt.
- Die Lösungen sollen in diesem IPython Notebook realisiert werden, wobei Teilaufgaben und Zwischenergebnisse ausgegeben bzw. visualisiert werden sollen.
- Für die Abgabe sollen Sie dieses IPython Notebook und ggf. zugehörige Dateien in ein **Ziparchiv** packen und im Ilias hochladen. Das Ziparchiv soll nach folgendem Muster benannt werden:
`UebungXX_Nachname1_Nachname2_Nachname3.zip`, wobei die Nachnamen in alphabetischer Reihenfolge angegeben und Umlaute ggf. ersetzt werden sollen. Bei Nichtbefolgung dieser Vorgabe können Punkte abgezogen werden.

---

## Aufgabe 1: K-Means++

a) Implementieren Sie eine Funktion `k_means(data, k)`, die den K-means-Algorithmus auf ein Numpy-Array mit Objektkoordinaten anwendet und die Clusterzuordnung jedes Punktes sowie die Clusterzentren zurückliefert. Verwenden Sie keine Bibliotheken wie `sklearn`, sondern setzen Sie den in der Vorlesung vorgestellten Algorithmus um.

b) Ergänzen Sie Ihre Funktion um einen weiteren Parameter `initialization`, über den spezifiziert wird, wie die Cluster initialisiert werden. Implementierne Sie folgende Fälle:
- `initialization == 'random'`: zufällige Cluster-Initialisierung zwischen dem Minimum und Maximum der Daten
- `initialization == '++'`: Cluster-Initialisierung von L-Means++ (siehe https://en.wikipedia.org/wiki/K-means%2B%2B)

c) Lesen Sie die Datei `clustering.csv` ein und wenden Sie K-Means für $k=6$ an. 

Lassen Sie jedes der folgenden Clusterings mindestens fünf Mal durchlaufen und visualisieren Sie die jeweiligen Ergebnisse als Scatterplots, in denen erkennbar ist, welcher Punkt zu welchem Cluster gehört.

1. Ihre Implementierung mit zufälliger Initialisierung 
2. Die Implementierung aus `sklearn` mit zufälliger Initialisierung
3. Ihre Implementierung mit K-Means++-Initialisierung
4. Die Implementierung aus `sklearn` mit K-Means++-Initialisierung

---
## Aufgabe 2: Dichte-basiertes Clustering (0,5+0,5+4 Punkte)

a) Beschreiben Sie kurz, wann dichte-basierte Clusteringverfahren eingesetzt werden und welche Vorteile und Nachteile es gibt.

b) Erklären Sie den Unterschied zwischen dichte-erreichbaren und dichte-verbundenen Punkten (z.B. anhand einer Skizze). 

c) Implementieren Sie die Methoden `DBSCAN`, `expandCluster` und `regionQuery` in der Datei `DBSCAN.py`.

d) Wenden Sie die zuvor implementierte Methode `DBSCAN` auf die Punkte aus der Datei `clustering.csv` an. Wählen Sie passende Werte für `min_pts` und `eps` und visualisieren Sie das Ergebnis.